**Это моё решение задачи в качестве сабмишена для финального проекта специализации 
на курсере по введение в машинное обучение от вышки и Яндекса**

Для начала просто извлечём данные: сделаем это при помощи ранее выбранных признаков. Поскольку цели показать  большой балл на лидерборде у меня нет, то обойдёмся этим неполным набором признаков.

In [15]:
import pandas as pd
train_data = pd.read_csv('./features.csv', index_col='match_id')
test_data = pd.read_csv('./features_test.csv', index_col='match_id')
train_data.head(5)

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,4,2,2,-52.0,2874,1,1796,0,51,0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,4,3,1,-5.0,2463,1,1974,0,63,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,4,3,1,13.0,2130,0,0,1830,0,63
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,4,2,0,27.0,1459,0,1920,2047,50,63
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,3,3,0,-16.0,2449,0,4,1974,3,63


Предлагаю посмотреть более детально какие признаки мы имеем

In [16]:
features = []
for label in train_data.columns:
    features.append(label)
print(features)

['start_time', 'lobby_type', 'r1_hero', 'r1_level', 'r1_xp', 'r1_gold', 'r1_lh', 'r1_kills', 'r1_deaths', 'r1_items', 'r2_hero', 'r2_level', 'r2_xp', 'r2_gold', 'r2_lh', 'r2_kills', 'r2_deaths', 'r2_items', 'r3_hero', 'r3_level', 'r3_xp', 'r3_gold', 'r3_lh', 'r3_kills', 'r3_deaths', 'r3_items', 'r4_hero', 'r4_level', 'r4_xp', 'r4_gold', 'r4_lh', 'r4_kills', 'r4_deaths', 'r4_items', 'r5_hero', 'r5_level', 'r5_xp', 'r5_gold', 'r5_lh', 'r5_kills', 'r5_deaths', 'r5_items', 'd1_hero', 'd1_level', 'd1_xp', 'd1_gold', 'd1_lh', 'd1_kills', 'd1_deaths', 'd1_items', 'd2_hero', 'd2_level', 'd2_xp', 'd2_gold', 'd2_lh', 'd2_kills', 'd2_deaths', 'd2_items', 'd3_hero', 'd3_level', 'd3_xp', 'd3_gold', 'd3_lh', 'd3_kills', 'd3_deaths', 'd3_items', 'd4_hero', 'd4_level', 'd4_xp', 'd4_gold', 'd4_lh', 'd4_kills', 'd4_deaths', 'd4_items', 'd5_hero', 'd5_level', 'd5_xp', 'd5_gold', 'd5_lh', 'd5_kills', 'd5_deaths', 'd5_items', 'first_blood_time', 'first_blood_team', 'first_blood_player1', 'first_blood_playe

Отлично! Теперь мы видим перед глазами набор признаков, которые у нас есть. Сразу же хочется понять как их можно объединить... С одной стороны мы видим всевозможные статистики по каждому из героев. Много ли от этого толка? Не уверен, потому что игра командная...
<br>На самом деле надо немного объянить типичную ситауцию в доте - есть мидер, керри, саппорты и оффлейнер. Как правило саппорты не имеют ни денег, ни опыта, покупают вещи на команду(варды, хилки, телепорты и так далее). Мидеры стоят 1 на 1, имеют много опыта и их задача на первые 5 минут обыграть вражеского мидера, получить много опыта и задоминировать линию и как правило, мидеры задают динамику боя - быстрее всех получают сильную способность(ультимейт) и гангают другие лайны, убивая врагов. Задача керри - фармить деньги, убивая крипов, чтобы в конце боя убить всех остальных - как правило такие герои критичны к крипстату. Ну и наконец есть оффлейнеры - врываторы, инициаторы -  они стоят против 2-3 героев... Их задача получить как можно больше опыта, при этом желательно не умирая.
<br>Этакие правило дотера - ботинки на первой минуте - плохо, нет куры - плохо, варды не стоят - саппорты раки. Фид врагов тоже понятен.
<br>Понятно, что в целом, из-за командности игры можно ввести признак, который будет показывать общие показатели команды... Хотя он и будет линейно зависим от остальных.
<br>Теперь что касается результатов матча - у нас есть статистики по зданиям и победитель встречи... Целевой переменно тут конечно является "radiant_win"

Теперь давайте уберём лишние признаки

In [20]:
#запишем нашу целевую переменную
y_train = train_data['radiant_win']
#запомним лишние данные
towers_dict = ['duration', 'tower_status_radiant', 'tower_status_dire', 'barracks_status_radiant', 
               'barracks_status_dire', 'radiant_win']
towers_df = data[towers_dict]

#Да, мы записали эти данные отдельно, но ещё не убрали их из трейна
train_df = train_data.drop(towers_dict, axis=1)
train_df.head(5)

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,0,35.0,103.0,-84.0,221.0,3,4,2,2,-52.0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,0,-20.0,149.0,-84.0,195.0,5,4,3,1,-5.0
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,1,-39.0,45.0,-77.0,221.0,3,4,3,1,13.0
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,0,-30.0,124.0,-80.0,184.0,0,4,2,0,27.0
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,0,46.0,182.0,-80.0,225.0,6,3,3,0,-16.0


Теперь всё хорошо, но по факту наши признаки нужно обрабатывать и обрабатывать. Да, большинство из них числовые, но не все, да и значения  некоторых остутствуют

In [25]:
#Первым делом посчитаем в каких столбцах у нас сколько значений
data_nulls = {}
print(len(y_train))
for label in train_df.columns:
    if train_df[label].count() < 97230:
        data_nulls[label]=[train_df[label].count(), test_data[label].count()]
print(data_nulls) 

97230
{'radiant_first_ward_time': [95394, 16847], 'first_blood_player1': [77677, 13625], 'first_blood_player2': [53243, 9411], 'dire_flying_courier_time': [71132, 12653], 'dire_bottle_time': [81087, 14335], 'radiant_courier_time': [96538, 17050], 'radiant_bottle_time': [81539, 14282], 'dire_first_ward_time': [95404, 16914], 'first_blood_time': [77677, 13625], 'radiant_flying_courier_time': [69751, 12292], 'first_blood_team': [77677, 13625], 'dire_courier_time': [96554, 17047]}


Давайте возьмём 2 признака с наибольшим числом пропусков и попробуем про них что-то сказать:
<br>1)<b>"first_blood_player2"</b> - первый килл был сделан без ассистов 
<br>2)<b>"radiant_flying_courier_time"</b> - кура не нужна =D Учитывая что доступ к ней с 3+ минуты

In [26]:
#Уберём пропуски!
train_df = train_df.fillna(0).astype('int')
test_df = test_data.fillna(0).astype('int')

Теперь самое весёлое - время <b>Обучения</b>!

In [28]:
%%time
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from tqdm import tqdm
kf = KFold(n_splits=5, shuffle=True)
tree_amount = [10, 20, 30]
for n_tree in tqdm(tree_amount):
    clf = GradientBoostingClassifier(n_estimators=n_tree)
    print(cross_val_score(clf, train_df, y_train, scoring='roc_auc', cv=kf).mean())
    


 33%|███▎      | 1/3 [00:30<01:00, 30.26s/it]

0.66503769164


 67%|██████▋   | 2/3 [01:23<00:37, 37.17s/it]

0.681717876978


100%|██████████| 3/3 [02:42<00:00, 49.60s/it]

0.689361025689
CPU times: user 2min 39s, sys: 2.28 s, total: 2min 41s
Wall time: 2min 42s


Итак, что делать, если число деревьев растёт? Ответ как мне кажется тут довольно прост. Как мы видим из результатов обучения над прошлым множеством, то можно сделать вывод, что максимум пока не достигнут, соотвественно можно продолжать тюнить нашу модель. Поскольку мы используем кросс-валидацию, то переобучения быть не должно. Вопрос о том, как ускорить наш алгоритм? Мне кажется, что одна минута - не такой долгий срок. Рост времени от кол-ва деревьев является линейным.
<br>Ответы на вопрос про ускорение уже есть внизу - можно уменьшить обучающую выборку - деревья малочувствительны к размеру, можно уменьшить глубину, поскольку в градиентном бустинге в отличие от случайного леса нам важна ширина. Есть стохастический градиентный бустинг. Там случайные подвыборки... Наверное можно посмотреть на фичи и выкинуть признаки, которые слабо коррелируют с целевой переменной.

<b>Часть вторая. Логистическая регрессия</b>

Итак, у нас есть уже предобработанный сет. Теперь его надо нормализовать и применить к нему логистическую регрессию. Давайте это и сделаем!
<br>Чтобы подобрать параметр C можно конечно воспользоваться гридом, но поскольку от нас требуют повторить прошлую задачу, то давайте сделаем это вручную

In [30]:
%%time
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import numpy as np

X_train = StandardScaler().fit_transform(train_df)
C_space = np.logspace(-3, 2, 10)

for c in tqdm(C_space):
    clf = LogisticRegression(C=c)
    print(c, cross_val_score(clf, X_train, y_train, scoring='roc_auc', cv=kf).mean())

  0%|          | 0/10 [00:00<?, ?it/s] 10%|█         | 1/10 [00:10<01:31, 10.15s/it]

0.001 0.715929648986


 20%|██        | 2/10 [00:20<01:21, 10.18s/it]

0.0035938136638 0.716345477919


 30%|███       | 3/10 [00:32<01:14, 10.69s/it]

...

7.74263682681 0.716517092768


 90%|█████████ | 9/10 [02:07<00:15, 15.44s/it]

27.8255940221 0.716285030612


100%|██████████| 10/10 [02:24<00:00, 15.83s/it]

100.0 0.716615811201
CPU times: user 1min 52s, sys: 47.1 s, total: 2min 39s
Wall time: 2min 29s


In [32]:
%%time
#попробуем найти более точное значение C
C_space = np.linspace(0.1, 1, 10)

for c in tqdm(C_space):
    clf = LogisticRegression(C=c)
    print(c, cross_val_score(clf, X_train, y_train, scoring='roc_auc', cv=kf).mean())

  0%|          | 0/10 [00:00<?, ?it/s] 10%|█         | 1/10 [00:17<02:35, 17.22s/it]

0.1 0.716448217376


 20%|██        | 2/10 [00:35<02:19, 17.47s/it]

0.2 0.716327265649


 30%|███       | 3/10 [00:53<02:03, 17.70s/it]

...

0.8 0.716494778504


 90%|█████████ | 9/10 [02:31<00:16, 16.30s/it]

0.9 0.716312764717


100%|██████████| 10/10 [02:44<00:00, 15.28s/it]

1.0 0.716593745544
CPU times: user 1min 59s, sys: 59.1 s, total: 2min 58s
Wall time: 2min 44s


В целом можно увидеть, что искать С более точно нет смысла. Причиной, почему наш скор так скачет, скорее всего является то, что не ни фиксировали разбиение(Random State) при кросс-валидации... Поэтому без зазрения совести можно взять C = 0.5.
<br>Соотвественно качество при нём примерно 0.716. Надо заметить, что оно площадь под рок-кривой получилась больше, чем при градиентном бустинге, хотя по идее деревья дольжны давать большую точность. Скорее всего это обусловлено тем, что в случае Градиентного бустинга мы просто недообучились и можно было увеличивать число деревьев(разница между 20 и 30 была порядка 0.01, что довольно много)
<br>И если говорить про скорость - регрессия безусловно быстрее